In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import keras
from math import sqrt
from matplotlib import pyplot
import numpy as np
import pandas as pd
import tensorflow as tf

Using TensorFlow backend.


In [2]:
def fit_lstm(Dx,Dy, nb_epoch, neurons,load_dir=None,load_old_model=False,lr=0.000001,start_epoch=0):
    if (load_old_model):
        print("loading old model.")
        model=keras.models.load_model(load_dir)
    else:
        model = Sequential()
        model.add(LSTM(neurons, batch_input_shape=(1, 1,Dx.shape[2]), stateful=True))
        model.add(Dense(units=16,activation = 'relu'))
        model.add(Dense(units=1,activation = 'relu'))    
    
    
    opt = keras.optimizers.Adam(lr = lr)
    model.compile(loss='mean_squared_error', optimizer=opt,metrics=['accuracy'])
    
    X,y = data_shuffle(Dx,Dy)
    sample_weight = y*len(y)/y.sum()+1
    for i in range(start_epoch+1,start_epoch+nb_epoch):
        print("epoch"+str(i))
        model.fit(X, y, epochs=1, batch_size=1, verbose=1, shuffle=False,sample_weight=sample_weight)
        model.reset_states()
        saveStr = './model/25hz/keras_LSTM_combinedData_sample_'+str(neurons)+'_i'+str(i)+'.hdf5'
        model.save(saveStr)
        if (i%10==0):
            X,y = data_shuffle(Dx,Dy)
            predict = model.predict(X,batch_size=1,verbose=1)
            predict = predict.reshape([len(predict)])
            print(predict.sum())
            if (predict.sum()<(y.sum()/10)):
                sample_weight = y*len(y)/y.sum()*1+1
            else:
                sample_weight = y*len(y)/y.sum()*2+1 + predict*len(predict)/predict.sum()
        
    return model

In [3]:
def data_shuffle(x,y,gap=150,nonActGap=500):
    
    label_index= np.where(y==1)[0]
    label_index = np.append(label_index,[len(y)-1])
    label_index = np.append([0],label_index)
    segments=np.zeros([0,2],dtype=int)
    last_seg = 0

    for i in range(len(label_index) -1):
        if ((label_index[i+1]-label_index[i])>gap):
            num_seg = int((label_index[i+1]-label_index[i])/nonActGap)+2
            seg_gap = int((label_index[i+1]-label_index[i])/num_seg)
            for j in range(num_seg):
                if j == num_seg-1:  # last segment
                    seg = label_index[i+1]
                else:
                    seg = seg_gap*(j+1)+label_index[i]
                segments = np.append(segments,[[last_seg,seg]],axis=0)
                last_seg=seg
    
    newSeg = np.arange(len(segments))
    np.random.shuffle(newSeg)
    if (len(x.shape)==3):
        newx = np.zeros([0,x.shape[1],x.shape[2]])
    else:
        newx = np.zeros([0,x.shape[1]])
    newy = np.zeros([0])
    #print("number of segments:"+str(len(newSeg)))
    for i in range(len(newSeg)):
        newx = np.append(newx,x[segments[newSeg[i]][0]:segments[newSeg[i]][1]],axis=0)
        newy = np.append(newy,y[segments[newSeg[i]][0]:segments[newSeg[i]][1]],axis=0)
    return newx,newy

In [4]:
Dx1 = np.loadtxt('./data/PSDSData (17_20)_raw.csv',delimiter=',')
Dy1 = np.loadtxt('./data/push_finish(17_20)_1.csv',delimiter=',')
Dx2 = np.loadtxt('./data/PSDSData (30)_raw.csv',delimiter=',')
Dy2 = np.loadtxt('./data/push_30.csv',delimiter=',')
Dx3 = np.loadtxt('./data/PSDSData (32)_raw.csv',delimiter=',')
Dy3 = np.loadtxt('./data/push_32.csv',delimiter=',')
DataNonAct = np.loadtxt('./data/NonActivityData.csv',delimiter=',')
DataNonActY = np.zeros_like(DataNonAct[:,1])

In [5]:
DataNonAct_shuffle,DataNonActY_shuffle=data_shuffle(DataNonAct,DataNonActY,nonActGap=1000)
DataNonAct_sample =DataNonAct_shuffle[0:int(len(DataNonAct_shuffle)/10)]
DataNonActy_sample =DataNonActY_shuffle[0:int(len(DataNonAct_shuffle)/10)]

In [6]:
DataNonAct_sample.shape

(157223, 9)

In [7]:
Dx3.shape

(134823, 13)

In [8]:
Dx3 = Dx3[:,0:9]

In [9]:
Dx3.shape

(134823, 9)

In [10]:
useSample =True

In [11]:
Datax = np.empty([0,9])
Datax = np.append(Datax,Dx1,axis=0)
Datax = np.append(Datax,Dx2,axis=0)
Datax = np.append(Datax,Dx3,axis=0)
if useSample==True:
    Datax = np.append(Datax,DataNonAct_sample,axis=0)
else:
    Datax = np.append(Datax,DataNonAct,axis=0)

Datay = np.empty([0])
Datay = np.append(Datay,Dy1,axis=0)
Datay = np.append(Datay,Dy2,axis=0)
Datay = np.append(Datay,Dy3,axis=0)
if useSample==True:
    Datay = np.append(Datay,DataNonActy_sample,axis=0)    
else:
    Datay = np.append(Datay,DataNonActY,axis=0)



In [12]:
Datax.shape

(508752, 9)

In [13]:
if useSample==True:
    np.savetxt('./data/train/CombineDataX_sample.csv',Datax,delimiter=',')
    np.savetxt('./data/train/CombineDatay_sample.csv',Datay,delimiter=',')
else:
    
    np.savetxt('./data/train/CombineDataX.csv',Datax,delimiter=',')
    np.savetxt('./data/train/CombineDatay.csv',Datay,delimiter=',')

In [16]:
Datax=np.loadtxt('./data/train/combineDatax.csv',delimiter=',')
Datay=np.loadtxt('./data/train/combineDatay.csv',delimiter=',')

OSError: ./data/train/combineDatax.csv not found.

In [14]:
Datax = Datax[:,3:9].reshape(Datax.shape[0],1,6)
Datax[:,:,3:6] = np.arccos(Datax[:,:,3:6]/10.0)

In [15]:
Datay.shape

(508752,)

In [16]:
Datax.shape

(508752, 1, 6)

In [17]:
lstm_model=fit_lstm(Datax,Datay,2,64)

W1217 15:05:54.103880 140236459927360 deprecation_wrapper.py:119] From /home/xing/anaconda3/envs/tf_cpu/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1217 15:05:54.248898 140236459927360 deprecation_wrapper.py:119] From /home/xing/anaconda3/envs/tf_cpu/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1217 15:05:54.300055 140236459927360 deprecation_wrapper.py:119] From /home/xing/anaconda3/envs/tf_cpu/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1217 15:05:54.945503 140236459927360 deprecation_wrapper.py:119] From /home/xing/anaconda3/envs/tf_cpu/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.comp

epoch1


W1217 15:05:56.229792 140236459927360 deprecation_wrapper.py:119] From /home/xing/anaconda3/envs/tf_cpu/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W1217 15:05:56.286505 140236459927360 deprecation_wrapper.py:119] From /home/xing/anaconda3/envs/tf_cpu/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Epoch 1/1
508751/508751 [==============================] - 816s 2ms/step - loss: 0.2867 - acc: 0.8877


In [48]:
start_epoch=200
model_dir = './models/25hz/Keras_model/keras_LSTM_combinedData_'+str(64)+'_i'+str(start_epoch)+'.hdf5'

lstm_model=fit_lstm(Datax,Datay,500,64,load_dir=model_dir,load_old_model=True,lr=0.000001,start_epoch=start_epoch)

loading old model.
epoch201
Epoch 1/1
508751/508751 [==============================] - 775s 2ms/step - loss: 0.1380 - acc: 0.9468
epoch202
Epoch 1/1
508751/508751 [==============================] - 772s 2ms/step - loss: 0.1293 - acc: 0.9436
epoch203
Epoch 1/1
508751/508751 [==============================] - 792s 2ms/step - loss: 0.1266 - acc: 0.9421
epoch204
Epoch 1/1
508751/508751 [==============================] - 769s 2ms/step - loss: 0.1298 - acc: 0.9417
epoch205
Epoch 1/1
508751/508751 [==============================] - 785s 2ms/step - loss: 0.1331 - acc: 0.9404
epoch206
Epoch 1/1
508751/508751 [==============================] - 785s 2ms/step - loss: 0.1375 - acc: 0.9384
epoch207
Epoch 1/1
508751/508751 [==============================] - 807s 2ms/step - loss: 0.1380 - acc: 0.9364
epoch208
Epoch 1/1
508751/508751 [==============================] - 783s 2ms/step - loss: 0.1376 - acc: 0.9361
epoch209
Epoch 1/1
508751/508751 [==============================] - 773s 2ms/step - loss: 0.1

508751/508751 [==============================] - 626s 1ms/step - loss: 0.6962 - acc: 0.8842
epoch340
Epoch 1/1
508751/508751 [==============================] - 185s 364us/step
87128.52
epoch341
Epoch 1/1
508751/508751 [==============================] - 627s 1ms/step - loss: 0.7576 - acc: 0.8975
epoch342
Epoch 1/1
508751/508751 [==============================] - 626s 1ms/step - loss: 0.8097 - acc: 0.9019
epoch343
Epoch 1/1
508751/508751 [==============================] - 627s 1ms/step - loss: 0.7556 - acc: 0.8972
epoch344
Epoch 1/1
508751/508751 [==============================] - 627s 1ms/step - loss: 0.7386 - acc: 0.8980
epoch345
Epoch 1/1
508751/508751 [==============================] - 626s 1ms/step - loss: 0.7753 - acc: 0.8953
epoch346
Epoch 1/1
508751/508751 [==============================] - 628s 1ms/step - loss: 0.8146 - acc: 0.8979
epoch347
Epoch 1/1
508751/508751 [==============================] - 629s 1ms/step - loss: 0.7507 - acc: 0.8980
epoch348
Epoch 1/1
508751/508751 [====

508751/508751 [==============================] - 626s 1ms/step - loss: 0.5063 - acc: 0.8920
epoch478
Epoch 1/1
508751/508751 [==============================] - 626s 1ms/step - loss: 0.5055 - acc: 0.8900
epoch479
Epoch 1/1
508751/508751 [==============================] - 627s 1ms/step - loss: 0.5053 - acc: 0.8872
epoch480
Epoch 1/1
508751/508751 [==============================] - 185s 364us/step
143384.73
epoch481
Epoch 1/1
508751/508751 [==============================] - 628s 1ms/step - loss: 0.5861 - acc: 0.8898
epoch482
Epoch 1/1
508751/508751 [==============================] - 626s 1ms/step - loss: 0.6162 - acc: 0.8854
epoch483
Epoch 1/1
508751/508751 [==============================] - 628s 1ms/step - loss: 0.6780 - acc: 0.8854
epoch484
Epoch 1/1
508751/508751 [==============================] - 627s 1ms/step - loss: 0.6182 - acc: 0.8869
epoch485
Epoch 1/1
508751/508751 [==============================] - 628s 1ms/step - loss: 0.6544 - acc: 0.8807
epoch486
Epoch 1/1
508751/508751 [===

508751/508751 [==============================] - 627s 1ms/step - loss: 0.5808 - acc: 0.8999
epoch616
Epoch 1/1
508751/508751 [==============================] - 627s 1ms/step - loss: 0.5875 - acc: 0.8998
epoch617
Epoch 1/1
508751/508751 [==============================] - 627s 1ms/step - loss: 0.5579 - acc: 0.9009
epoch618
Epoch 1/1
508751/508751 [==============================] - 627s 1ms/step - loss: 0.5528 - acc: 0.8966
epoch619
Epoch 1/1
508751/508751 [==============================] - 627s 1ms/step - loss: 0.5728 - acc: 0.8865
epoch620
Epoch 1/1
508751/508751 [==============================] - 185s 364us/step
68481.195
epoch621
Epoch 1/1
508751/508751 [==============================] - 627s 1ms/step - loss: 0.6236 - acc: 0.8857
epoch622
Epoch 1/1
508751/508751 [==============================] - 628s 1ms/step - loss: 0.6653 - acc: 0.8797
epoch623
Epoch 1/1
508751/508751 [==============================] - 629s 1ms/step - loss: 0.6392 - acc: 0.8902
epoch624
Epoch 1/1
508751/508751 [===

KeyboardInterrupt: 